# Systematic Analysis of Adversarial Errors

This notebook performs a systematic search for adversarial errors in surface codes of different distances. An adversarial error is a physical error that, due to the circuit structure and gate ordering, propagates into a logical error.

This analysis uses a classical Pauli propagator to perfectly model the noise-free error propagation. It iterates through:
- Code distances (d=3 and d=5)
- All 24 CNOT permutations for weight-4 stabilizers
- Pauli X and Z errors
- All possible injection points (on every weight-4 ancilla, after every CNOT in its stabilizer sequence)

The goal is to identify which combinations of these parameters are most likely to cause logical failures, providing a more comprehensive view of error propagation beyond the initial set of hook errors.

In [ ]:
# === 1. Imports and Setup ===
import itertools
import pprint

from zx_error_prop.propagation import count_logical_errors
from zx_error_prop.stim_interface import _simulate_hook_paulis_qasm
from zx_error_prop.surface_code import generate_rotated_surface_code, parse_qubit_map, find_neighboring_data_qubits

In [ ]:
# === 2. Main Analysis Function ===

def find_adversarial_errors(distance, permutations):
    print(f"--- Starting Adversarial Error Analysis for d={distance} ---")
    
    # 1. Build code properties for the given distance
    sc_map = generate_rotated_surface_code(distance)
    _, ancillas = parse_qubit_map(sc_map)
    connectivity = {anc['label']: find_neighboring_data_qubits(sc_map, *anc['matrix_coord']) for anc in ancillas}
    weight4_stabs = sorted([lab for lab, neigh in connectivity.items() if len(neigh) == 4], key=lambda s: (s[0], int(s[1:])))
    
    # 2. Define logical operators
    D2 = distance * distance
    z_logicals = [list(range(r * distance, r * distance + distance)) for r in range(distance)]
    x_logicals = [list(range(c, D2, distance)) for c in range(distance)]
    
    adversarial_configs = []
    ERROR_TYPES = ['X', 'Z']

    # 3. Main loop to test all configurations
    for i, perm in enumerate(permutations):
        if (i % 4 == 0):
             print(f"  Processing permutation {i+1}/{len(permutations)}: {perm}")
        
        for stab_label in weight4_stabs:
            for hook_step in range(4): # 4 CNOTs for a weight-4 stabilizer
                for error_type in ERROR_TYPES:
                    
                    # Simulate the error propagation
                    final_paulis = _simulate_hook_paulis_qasm(
                        distance=distance,
                        ordering=perm,
                        target_stab=stab_label,
                        hook_step=hook_step,
                        error_type=error_type,
                        weight4_stabs=weight4_stabs
                    )
                    
                    # Convert to dictionary format for error counting function
                    output_errors = {q: p for q, p in enumerate(final_paulis) if p != 'I'}
                    
                    # Count logical errors
                    z_errs, x_errs = count_logical_errors(output_errors, z_logicals, x_logicals)
                    
                    if z_errs > 0 or x_errs > 0:
                        result = {
                            "permutation": perm,
                            "stabilizer": stab_label,
                            "hook_step": hook_step,
                            "error_type": error_type,
                            "logical_errors": (z_errs, x_errs)
                        }
                        adversarial_configs.append(result)
                        
    print(f"--- Analysis for d={distance} complete. Found {len(adversarial_configs)} adversarial configurations. ---")
    return adversarial_configs


In [ ]:
# === 3. Run Analysis and Display Results ===

all_perms = list(itertools.permutations([0, 1, 2, 3]))

print("Running for d=3. This may take several minutes.")
d3_results = find_adversarial_errors(3, all_perms)

print("\n--- d=3 Adversarial Configurations Found ---")
pprint.pprint(d3_results)

print("\nNOTE: A full d=5 analysis would be very time-consuming and is not run by default.")
print("To run it, you can call: find_adversarial_errors(5, all_perms)")